In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


In [3]:
pixel_means = X_train.mean(axis = 0, keepdims=True)
pixel_stds = X_train.std(axis = 0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = ((X_valid - pixel_means) / pixel_stds)
X_test_scaled = ((X_test - pixel_means) / pixel_stds)

In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, activation = "elu", 
                                                 kernel_initializer = "he_normal", 
                                                 kernel_regularizer = keras.regularizers.l2(0.01))



In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation = 'softmax')
])

In [7]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 6s 3ms/step - loss: 1.6389 - accuracy: 0.8116 - val_loss: 0.7182 - val_accuracy: 0.8324
Epoch 2/2
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7185 - accuracy: 0.8280 - val_loss: 0.6828 - val_accuracy: 0.8382


MC dropout

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

In [9]:
y_probas = np.stack([model(X_test_scaled, training = True) for sample in range(100)])


In [10]:
y_probas.shape

(100, 10000, 10)

In [11]:
y_proba = y_probas.mean(axis = 0)

In [12]:
np.round(model.predict(X_test_scaled[:1]), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],
      dtype=float32)

In [13]:
np.round(y_probas[:, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.26, 0.01, 0

In [14]:
y_std = y_probas.std(axis = 0)
np.round(y_std[:1], 2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [15]:
y_pred = np.argmax(y_proba, axis=1)

In [16]:
accuracy = np.sum(y_pred == y_test)/ len(y_test)
accuracy

0.82